In [ ]:
# Retreive data

# 1. Create EXCEL from official PDF using ILovePDF OCR
# 2. Open Excel and remove metadata and titles
# 3. Export Excel to CSV
# 4. Fix CSV adding new column to fill al the rows (Non-answered second option are empty). Replaced my "NO" to 0 and "SI" to 1
# 5. Open CSV with pandas

,SECRETARÍA DE ESTADO DE EDUCACIÓN,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,"MINISTERIO\nDE EDUCACIÓN, FORMACIÓN PROFESIONA...",NaN,NaN,DIRECCIÓN GENERAL DE PLANIFICACIÓN Y GESTIÓN E...,NaN,NaN
1,PROGRAMA AUXILIARES DE CONVERSACIÓN ESPAÑOLES ...,NaN,NaN,NaN,NaN,NaN
2,20 de marzo de 2025\n1,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,"APELLIDOS, NOMBRE",NaN,DOCUMENTO DE\nIDENTIDAD,NaN,PAÍS 1,PAÍS 2


In [ ]:
import os
from pathlib import Path

NUMBER_OF_COLUMNS = 4
SEPARATOR = ';'
ADD_MESSAGE = 'No contestado'

fp = Path('lista-admitidos-2025-2026.csv')
fixed_fp = f"{fp.stem}-fixed{fp.suffix}"

new_lines = []

with open(fp, 'r') as f:
    for line in f:
        objs = line.split(SEPARATOR)

        for i, column in enumerate(objs):
            # if is empty but is not the last column, remove it
            if column == '' and i < NUMBER_OF_COLUMNS - 1:
                objs.pop(i)

        # Ensure exactly NUMBER_OF_COLUMNS elements
        while len(objs) < NUMBER_OF_COLUMNS:
            objs.append(ADD_MESSAGE)


        new_lines.append(SEPARATOR.join(objs))

# Save the corrected file
with open(fixed_fp, 'w', encoding='utf-8', newline='') as f:
    f.writelines(new_lines)

print(f"Fixed file saved as: {fixed_fp}")


Fixed file saved as: lista-admitidos-2025-2026-fixed.csv


In [39]:
# Load dataframe
import pandas as pd
df = pd.read_csv('lista-admitidos-2025-2026-fixed.csv', sep=';')
df.head()

,nombre,dni,primera_opción,segunda_opción
0,"Abad Torres, Sergio",***8607**,Reino Unido,Irlanda
1,"Abaso Bellido, Marina",***9122**,Australia,Nueva Zelanda
2,"Abatan Dopazo, Yasmina",***1487**,Francia,Bélgica
3,"Abellan Perez, Sofia",***5081**,Noruega,Irlanda
4,"Abello Trigueros, Fátima",***9861**,Canadá,NaN


In [ ]:
# Data Analysis I

# Count total occurrences for primera_opción and segunda_opción
primera_counts = df["primera_opción"].value_counts()
segunda_counts = df["segunda_opción"].value_counts()

# Merge both counts into a single DataFrame
country_counts = pd.DataFrame({
    "Primera Opción": primera_counts,
    "Segunda Opción": segunda_counts
}).fillna(0)  # Fill NaN with 0

# Add a new column for total occurrences (first + second choice)
country_counts["Total"] = country_counts["Primera Opción"] + country_counts["Segunda Opción"]

# Correlation Analysis: Convert categorical countries to numerical encoding
df_encoded = df.copy()
df_encoded["primera_opción"] = df_encoded["primera_opción"].astype("category").cat.codes
df_encoded["segunda_opción"] = df_encoded["segunda_opción"].astype("category").cat.codes
correlation = df_encoded["primera_opción"].corr(df_encoded["segunda_opción"])

# Filter for specific cases (Alemania and Austria)
specific_countries = country_counts.loc[country_counts.index.isin(["Alemania", "Austria"])]

# Display results
print("\n=== Total counts per country ===")
print(country_counts)

print(f"\n=== Correlation between primera_opción and segunda_opción: {correlation:.2f} ===")

print("\n=== Specific counts for Alemania and Austria ===")
print(specific_countries)


=== Total counts per country ===
               Primera Opción  Segunda Opción  Total
Alemania                  107              29    136
Australia                  24              24     48
Austria                    33              80    113
Brasil                      3               4      7
Bélgica                    58             129    187
Canadá                     34              65     99
EE. UU.                   115              53    168
Francia                   248              62    310
Irlanda                   217             240    457
Italia                     60               1     61
Malta                      27              46     73
Noruega                    35              41     76
Nueva Zelanda              30              31     61
Portugal                   12               3     15
Reino Unido               269             142    411
Suiza                      10              13     23
Tailandia                  16              15     31

=== Correla

In [42]:
# Data Analysis II
plazas_previstas = {
    'Alemania': [125, 'DE'],
    'Austria': [40, 'DE'],
    'Suiza': [3, 'DE'],
    'Bélgica': [15, 'FR'],
    'Canadá': [1, 'FR'],
    'Francia': [410, 'FR'],
    'Suiza': [1, 'FR'],
    'Australia': [1, 'EN'],
    'Canadá': [4, 'EN'],
    'Estados Unidos': [50, 'EN'],
    'Irlanda': [43, 'EN'],
    'Malta': [5, 'EN'],
    'Noruega': [5, 'EN'],
    'Nueva Zelanda': [6, 'EN'],
    'Reino Unido': [130, 'EN'],
    'Tailandia': [2, 'EN'],
    'Italia': [20, 'IT'],
    'Brasil': [3, 'PT'],
    'Portugal': [5, 'PT']
}

In [45]:
# Analysis III

# Merge both counts
total_selections = primera_counts.add(segunda_counts, fill_value=0)

# Create a DataFrame for comparison
analysis_df = pd.DataFrame(plazas_previstas).T
analysis_df.columns = ["Plazas Disponibles", "Grupo de Idioma"]

# Merge selection counts
analysis_df["Primera Opción"] = primera_counts
analysis_df["Segunda Opción"] = segunda_counts
analysis_df["Total Demand"] = total_selections
analysis_df["Available Slots"] = analysis_df["Plazas Disponibles"] - analysis_df["Total Demand"]

# Fill NaN values with 0
analysis_df = analysis_df.fillna(0)

# Print results
print("\n=== Analysis of Available Slots vs. Demand ===")
print(analysis_df)


=== Analysis of Available Slots vs. Demand ===
                Plazas Disponibles Grupo de Idioma  Primera Opción  \
Alemania                       125              DE           107.0   
Austria                         40              DE            33.0   
Suiza                            1              FR            10.0   
Bélgica                         15              FR            58.0   
Canadá                           4              EN            34.0   
Francia                        410              FR           248.0   
Australia                        1              EN            24.0   
Estados Unidos                  50              EN             0.0   
Irlanda                         43              EN           217.0   
Malta                            5              EN            27.0   
Noruega                          5              EN            35.0   
Nueva Zelanda                    6              EN            30.0   
Reino Unido                    130        

/var/folders/hj/dlhls6dd0ws_cwn890w5nydw0000gn/T/ipykernel_4037/2963230740.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  analysis_df = analysis_df.fillna(0)


In [46]:
# Data Analysis IV

print(f'Ordenar países por disponibilidad de plazas', analysis_df.sort_values(by='Available Slots', ascending=False))

Ordenar países por disponibilidad de plazas                 Plazas Disponibles Grupo de Idioma  Primera Opción  \
Francia                        410              FR           248.0   
Estados Unidos                  50              EN             0.0   
Brasil                           3              PT             3.0   
Portugal                         5              PT            12.0   
Alemania                       125              DE           107.0   
Suiza                            1              FR            10.0   
Tailandia                        2              EN            16.0   
Italia                          20              IT            60.0   
Australia                        1              EN            24.0   
Nueva Zelanda                    6              EN            30.0   
Malta                            5              EN            27.0   
Noruega                          5              EN            35.0   
Austria                         40            